# Data Preparation and Customer Analytics
# Task 1

Conduct analysis on client's transaction dataset and identify customer purchasing behaviours to generate insights and provide commercial recommendations.

## Background Information for Task

We need to present a strategic recommendation to Julia that is supported by data which she can then use for the upcoming category review however to do so we need to analyse the data to understand the current purchasing trends and behaviours. The client is particularly interested in customer segments and their chip purchasing behaviour. Consider what metrics would help describe the customers’ purchasing behaviour.

## Work FLow

**1. Examine transaction data;** 

    - Look for inconsistencies
    - missing data across the dataset
    - outliers
    - correctly identify category items
    - numeric data across all tables.
    
    If anomalies are determined, make necessary changes and save it
    
**2. Examine customer data;**

    - Look for inconsistencies
    - missing data across the dataset
    - outliers
    - correctly identify category items
    - numeric data across all tables.

After completing the above tasks, merge the two datasets so it is ready for analysis.

**3. Data analysis and customer segments;**

    - define metrics
    - look at total sales
    - drivers of sales
    - where the highest sales are coming from
    - create charts and graphs
    - note interesting trends and insights
    
**4. Deep dive into customer segments;**

    - define recommendation from insights
    - determine which segments should be targeted
    - form overall conclusion based on analysis
    
**4. Save analysis**

    - save analysis and visuals

We will start this analysis by loading required libraries for analysis

In [1]:
# load libriaries
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

In [2]:
# import data
transaction_df = pd.read_csv("../data/QVI_transaction_data.csv")
customer_df = pd.read_csv("../data/QVI_purchase_behaviour.csv")

## Exploratory Data Analysis

In [6]:
# view the first 10 rows of the data
transaction_df.head(10)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8
5,43604,4,4074,2982,57,Old El Paso Salsa Dip Tomato Mild 300g,1,5.1
6,43601,4,4149,3333,16,Smiths Crinkle Chips Salt & Vinegar 330g,1,5.7
7,43601,4,4196,3539,24,Grain Waves Sweet Chilli 210g,1,3.6
8,43332,5,5026,4525,42,Doritos Corn Chip Mexican Jalapeno 150g,1,3.9
9,43330,7,7150,6900,52,Grain Waves Sour Cream&Chives 210G,2,7.2


In [7]:
# check the data types of each column
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            264836 non-null  int64  
 1   STORE_NBR       264836 non-null  int64  
 2   LYLTY_CARD_NBR  264836 non-null  int64  
 3   TXN_ID          264836 non-null  int64  
 4   PROD_NBR        264836 non-null  int64  
 5   PROD_NAME       264836 non-null  object 
 6   PROD_QTY        264836 non-null  int64  
 7   TOT_SALES       264836 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 16.2+ MB


We can observe that the `DATE` column has a data type of `int64` instead of a `datatime` format.

We will need to convert this from `int64` to `datetime`.